# PIPITS Fungal ITS-dedicated Pipeline

* DADA2 can, in theory, process ITS data. This is a dedicated alternative


### Dependencies ####

##### || PIPITS ||
* Follow instructions provided at: 
* https://github.com/hsgweon/pipits
* Note: all dependencies which require 'sudo' will already be met (i.e. don't bother running those commands... they won't work anyways)

##### || deML ||
* Follow instructions provided at: 
* https://github.com/grenaud/deML

##### || phyloseq ||
* conda install -c r-igraph 
* Rscript -e "source('http://bioconductor.org/biocLite.R');biocLite('phyloseq')" 

##### || FUNGuild ||
* download FUNGUild script:
* https://raw.githubusercontent.com/UMNFuN/FUNGuild/master/Guilds_v1.1.py

### Citations ###
* Gweon, H. S., Oliver, A., Taylor, J., Booth, T., Gibbs, M., Read, D. S., et al. (2015). PIPITS: an automated pipeline for analyses of fungal internal transcribed spacer sequences from the Illumina sequencing platform. Methods in ecology and evolution, 6(8), 973-980.

* Renaud, G., Stenzel, U., Maricic, T., Wiebe, V., & Kelso, J. (2014). deML: robust demultiplexing of Illumina sequences using a likelihood-based approach. Bioinformatics, 31(5), 770-772.

* McMurdie and Holmes (2013) phyloseq: An R Package for Reproducible Interactive Analysis and Graphics of Microbiome Census Data. PLoS ONE. 8(4):e61217

* Nguyen NH, Song Z, Bates ST, Branco S, Tedersoo L, Menke J, Schilling JS, Kennedy PG. 2016. FUNGuild: An open annotation tool for parsing fungal community datasets by ecological guild. Fungal Ecology 20:241–248.





###### Last Modified by R. Wilhelm on October 12th, 2017 ######


# Step 1: User Input

In [3]:
import os

# Provide the directory for your index and read files
ITS = '/home/roli/APHID_ITS/aphids/'

# Provide 
datasets = [['aphid',ITS,'mapping_file.txt']]

# Ensure your reads files are named accordingly (or modify to suit your needs)
readFile1 = 'read1.fq.gz'
readFile2 = 'read2.fq.gz'
indexFile1 = 'index_read1.fq.gz'
indexFile2 = 'index_read2.fq.gz'

# Example of metadata file
#Index1	Index2	Name
#AATTCAA	CATCCGG	RG1
#CGCGCAG	TCATGGT	RG2
#AAGGTCT	AGAACCG	RG3
#ACTGGAC	TGGAATA	RG4

## Again, for our pipeline Index1 typically is the reverse complement of the reverse barcode, while Index2 is the forward barcode.

# Step 2: Demultiplex Raw Reads

In [40]:
# Ignore all the 'conflict' errors. The reads are paired so the conflicts are bogus (i.e. it gives a warning everytime an barcode appears in multiple samples, but no pairs are duplicated)

for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    metadata = directory+dataset[2]
    index1 = directory+indexFile1
    index2 = directory+indexFile2
    read1 = directory+readFile1
    read2 = directory+readFile2
    
    # Make output directory
    %mkdir $directory/pipits_input/
    
    # Run deML   ## Note: you may get error involving 'ulimit'. If so, exit your notebook. Enter 'ulimit -n 9999' at the command line, then restart a new notebook.
    !deML -i $metadata -f $read1 -r $read2 -if1 $index1 -if2 $index2 -o $directory/pipits_input/$name

    # Remove unnecessary 'failed' reads and index files
    %rm $directory/pipits_input/*.fail.* $directory/pipits_input/unknown*

rm: cannot remove '/home/roli/FORESTs_BHAVYA/WoodsLake/raw_seq/ITS//pipits_input/unknown*': No such file or directory


# Step 3: Make Sample Mapping File (aka. 'readpairlist')

In [41]:
import glob, re
for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    
    # Remove Previously Prepended Name (PIPITS wanted something)
    for file in glob.glob(directory+"pipits_input/"+name+"_*"):
        new_name = re.sub(name+"_","",file)
        os.rename(file, new_name)
    
    # Rename files with with extension .fq (PIPITS is PICKY)
    for file in glob.glob(directory+"pipits_input/*.fq.gz"):
        new_name = re.sub(".fq.gz",".fastq.gz",file)
        os.rename(file, new_name)
    
    # Remove Unbinned Reads
    %rm $directory/pipits_input/unknown*        
    
    # Run PIPITS List Prep
    input_dir = directory+"pipits_input/"
    output_dir = directory+name+".readpairslist.txt"
    
    !pipits_getreadpairslist -i $input_dir -o $output_dir -f


# Step 4: Pre-process Data with PIPITS (merge and QC)

In [7]:
for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    
    input_dir = directory+"/"
    output_dir = directory+"pipits_prep/"
    readpairfile = directory+name+".readpairslist.txt"
    
    !pipits_prep -i $input_dir -o $output_dir -l $readpairfile


# Step 4: Extract Variable Region (**User Input Required**)

In [8]:
ITS_Region = "ITS1"

for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    
    input_file = directory+"pipits_prep/prepped.fasta"
    output_dir = directory+"pipits_funits/"
    
    !pipits_funits -i $input_file -o $output_dir -x $ITS_Region 


# Step 5: Cluster and Assign Taxonomy

In [ ]:
for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    
    input_file = directory+"pipits_funits/ITS.fasta"
    output_dir = directory+"PIPITS_final/"

    !pipits_process -i $input_file -o $output_dir --Xmx 20G
    

# Step 6: Push OTU Table through FUNGuild

In [ ]:
for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
    
    # Prepare PIPITS output for FUNGuild
    !pipits_funguild.py -i $directory/PIPITS_final/otu_table.txt -o $directory/PIPITS_final/otu_table_funguild.txt
   
    # Run FUNGuild
    !python /home/db/FUNGuild/Guilds_v1.1.py -otu $directory/PIPITS_final/otu_table_funguild.txt -db fungi -m -u

# Step 7: Import into R   (WORK IN PROGRESS)

In [7]:
## Setup R-Magic for Jupyter Notebooks
import rpy2
import pandas as pd
%load_ext rpy2.ipython
%R library(phyloseq)
%R library(ape)

def fix_biom_conversion(file):
    with open(file, 'r') as fin:
        data = fin.read().splitlines(True)
    with open(file, 'w') as fout:
        fout.writelines(data[1:])
        
for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
   
    # Import Taxonomy File
    !gunzip $directory"/PIPITS_final/assigned_taxonomy.txt.gz"
    tax_file = pd.read_csv(directory+"/PIPITS_final/assigned_taxonomy.txt", sep="\t")
    %R -i tax_file
    %R tax_file <- tax_file[sort(row.names(tax_file)),] #read names must match the count_table
    !gzip $directory"/PIPITS_final/assigned_taxonomy.txt"
    
    # Import Counts Data
    !gunzip $directory"/PIPITS_final/otu_table.txt.gz"
    fix_biom_conversion(directory+"/output/"+name+".counts.final.tsv") # The biom converter adds a stupid line that messes with the table formatting
    count_table = pd.read_csv(directory+"/PIPITS_final/otu_table.txt", sep="\t")
    %R -i count_table
    %R rownames(count_table) = count_table$X.OTU.ID   
    #%R count_table$X.OTU.ID <- NULL    
    %R count_table <- count_table[sort(row.names(count_table)),] #read names must match the tax_table
    !gzip $directory"/PIPITS_final/otu_table.txt"

    # Import Sample Data
    sample_file = pd.read_table(directory+metadata, keep_default_na=False)
    %R -i sample_file
    %R rownames(sample_file) = sample_file$X.SampleID   
    %R sample_file$X.SampleID <- NULL
    %R sample_file$LinkerPrimerSequence <- NULL  ## Clean-up some other stuff
    
    # Convert to Phyloseq Objects
    %R p_counts = otu_table(count_table, taxa_are_rows = TRUE)    
    %R p_tax = tax_table(tax_file)
    %R taxa_names(p_tax) <- rownames(tax_file) # phyloseq throws out rownames
    %R colnames(p_tax) <- colnames(tax_file) # phyloseq throws out colnames
    
    # Merge Phyloseq Objects
    %R p_final = phyloseq(p_counts, p_tax)

    # Save Phyloseq Object as '.rds'
    #output = directory+"/output/p_"+name+".pipits.final.rds"
    #%%R -i output
    #%%R saveRDS(p_final, file = output)
    
    # Confirm Output
    #%%R print(p_final)

In [11]:
%%R
print(head(tax_file))

# Step 7: Clean-up Intermediate Files and Final Outputs

In [ ]:
for dataset in datasets:
    name = dataset[0]
    directory = dataset[1]
   
    %rm -r $directory/pipits_prep/
    %rm -r $directory/pipits_funits/
    %rm -r $directory/pipits_input/
    
    del_me = directory+name+".readpairslist.txt"
    %rm $del_me